### 사이트에서 키워드(주유소명)으로 검색하기

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from urllib.parse import urlparse, quote
import folium
import pandas as pd
from glob import glob

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(2)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="SEARCH_MOD4"]').click()
"""search = driver.find_element_by_xpath('//*[@id="OS_NM"]')
time.sleep(1)
search.send_keys('우장산주유소')
search.submit()
text = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[2]').text
text"""
station_list = ['우장산주유소', '직영관문주유소', '화곡현대주유소', '신화곡주유소', '경인고속주유소', '목화주유소', '뉴신정주유소', '화곡역주유소', '마곡대성주유소', '방화대성주유소']
gasoline_list = []; diesel_list = []
for station in station_list:
    search = driver.find_element_by_xpath('//*[@id="OS_NM"]')
    time.sleep(1)
    search.clear()
    time.sleep(1)
    search.send_keys(station)
    search.submit()
    time.sleep(1)
    gasoline = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[2]').text
    diesel = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[3]/font').text
    time.sleep(1)
    gasoline_list.append(int(gasoline))
    diesel_list.append(int(diesel))

driver.quit()

In [3]:
print(gasoline_list, diesel_list)

[1358, 1358, 1326, 1315, 1266, 1264, 1243, 1243, 1365, 1365] [1187, 1187, 1126, 1115, 1072, 1064, 1043, 1043, 1185, 1175]


In [4]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [5]:
name_list=[]; tel_list=[]; addr_list=[]; lat_list=[]; lng_list=[]

for station in station_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + quote(station)
    result = requests.get(url,headers={'Authorization': "KakaoAK "+kmap_key}).json()
    name_list.append(result['documents'][0]['place_name'])
    tel_list.append(result['documents'][0]['phone'])
    addr_list.append(result['documents'][0]['road_address_name'])
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [6]:
station_data = pd.DataFrame({
    'station_name':station_list,
    'tel':tel_list,
    'address':addr_list,
    'gasoline_price':gasoline_list,
    'diesel_price':diesel_list,
    'lat':lat_list,
    'lng':lng_list
})
station_data

,station_name,tel,address,gasoline_price,diesel_price,lat,lng
0,우장산주유소,02-3664-7163,서울 강서구 강서로 292,1358,1187,37.551611,126.836582
1,직영관문주유소,02-3665-2051,서울 강서구 공항대로 314,1358,1187,37.557875,126.842367
2,화곡현대주유소,02-2607-1942,서울 강서구 강서로 151,1326,1126,37.540264,126.841200
3,신화곡주유소,02-2611-3161,서울 강서구 국회대로 71,1315,1115,37.527260,126.842889
4,경인고속주유소,02-2608-5151,서울 강서구 국회대로 225,1266,1072,37.529624,126.859795
5,목화주유소,02-2654-4564,서울 강서구 국회대로 251,1264,1064,37.530020,126.863062
6,뉴신정주유소,02-2601-0077,서울 강서구 곰달래로 207,1243,1043,37.531550,126.856042
7,화곡역주유소,02-2691-5151,서울 강서구 강서로 154,1243,1043,37.540625,126.841525
8,마곡대성주유소,02-3664-7781,서울 강서구 강서로 457,1365,1185,37.566279,126.840412
9,방화대성주유소,02-3663-7781,서울 강서구 양천로 176,1365,1175,37.572898,126.823436


In [7]:
min(station_data['gasoline_price'])

1243

In [8]:
map = folium.Map(location=[station_data['lat'].mean(), station_data['lng'].mean()], zoom_start=13)

for i in station_data.index:
    frame = f"{station_data['station_name'][i]}"
    popup = folium.Popup(frame, max_width=130)
    if station_data['gasoline_price'][i] == min(station_data['gasoline_price']):
        folium.Marker(
            location = [station_data['lat'][i], station_data['lng'][i]],
            tooltip=(f"휘발유:{station_data['gasoline_price'][i]}원, 가솔린:{station_data['diesel_price'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='red', icon='thumbs-up')
            ).add_to(map)
    else:
        folium.Marker(
            location = [station_data['lat'][i], station_data['lng'][i]],
            tooltip=(f"휘발유:{station_data['gasoline_price'][i]}원, 가솔린:{station_data['diesel_price'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='blue', icon='tint')
            ).add_to(map)
map

In [9]:
map = folium.Map(location=[station_data['lat'].mean(), station_data['lng'].mean()], zoom_start=13)

for i in station_data.index:
    frame = f"{station_data['station_name'][i]}"
    popup = folium.Popup(frame, max_width=130)
    if station_data['gasoline_price'][i] == min(station_data['gasoline_price']):
        folium.CircleMarker([station_data['lat'][i], station_data['lng'][i]],
                            radius=10,
                            color='#CD3181',
                            fill_color='#CD3181',
                            tooltip=(f"휘발유:{station_data['gasoline_price'][i]}원, 가솔린:{station_data['diesel_price'][i]}원"),
                            popup=popup
                            ).add_to(map)
    else:
        folium.CircleMarker([station_data['lat'][i], station_data['lng'][i]],
                            radius=10,
                            color='#4B89DC',
                            fill_color='#4B89DC',
                            tooltip=(f"휘발유:{station_data['gasoline_price'][i]}원, 가솔린:{station_data['diesel_price'][i]}원"),
                            popup=popup
                            ).add_to(map)
map